<a href="https://colab.research.google.com/github/tanatswamatar/2025DE245Prac2/blob/main/2025DE245Prac2_27124975.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Python for Data Engineering

This notebook demonstrates how to read data from a public API, process it to create useful information, and store the result in an SQLite database.

It assumes you have read [this article](https://www.startdataengineering.com/post/python-for-de/#extract--load-read-and-write-data-to-any-system).

**Acknowledgment**: Parts of this notebook were developed with the assistance of ChatGPT, an AI language model by OpenAI, to support instructional design and coding tasks related to data engineering concepts.

In [1]:
# Install required packages
!pip install requests

In [2]:
# Import libraries
import requests
import sqlite3
from datetime import datetime

In [3]:
# Read data from CoinGecko API
target = str(input("enter target currency"))
url_usd = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies=usd"
url_target = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin&vs_currencies="+ str(target)
response_usd= requests.get(url_usd)
data_usd = response_usd.json()
response_target = requests.get(url_target)
data_target = response_target.json()
#print(url_target)
#print(data_target)


enter target currencymxn


We will extract the Bitcoin price in USD and convert it to ZAR using an estimated exchange rate.
This is an example of turning raw data into useful **information**.

In [4]:
# Extract USD price and apply transformation
usd_price = data_usd["bitcoin"]["usd"]
target_price = data_target["bitcoin"][target]
#target_price = convert_usd_to_zar(usd_price)
timestamp = datetime.now().isoformat()

print(f"Bitcoin price in USD: {usd_price}")
print(f"Bitcoin price in {target}: {target_price}")
print(f"Timestamp: {timestamp}")


Bitcoin price in USD: 120354
Bitcoin price in mxn: 2231222
Timestamp: 2025-08-13T10:18:15.584891


In [5]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS bitcoin_prices (
          timestamp TEXT PRIMARY KEY,
          btc_price_usd REAL,
          target_currency TEXT,
          target_currency_price REAL
  )
  ''')

# Insert the new row

cursor.execute("INSERT INTO bitcoin_prices VALUES (?, ?, ?, ?)", (timestamp, usd_price, target, target_price))
conn.commit()
conn.close()

In [6]:
# Check if data was successfully written
conn = sqlite3.connect("bitcoin_prices.db")
cursor = conn.cursor()

# Show the last 5 rows
rows = cursor.execute("SELECT * FROM bitcoin_prices ORDER BY timestamp DESC LIMIT 5").fetchall()
for row in rows:
    print(row)

conn.close()

('2025-08-13T10:18:15.584891', 120354.0, 'mxn', 2231222.0)
('2025-08-13T10:16:17.687427', 120284.0, 'aed', 442047.0)
('2025-08-13T10:13:49.501160', 120222.0, 'hkd', 944268.0)
('2025-08-13T10:07:30.432890', 120126.0, 'jpy', 17688880.0)
('2025-08-13T10:06:51.701371', 120115.0, 'jpy', 17686075.0)
